In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import pickle
import hashlib
import json
import os
plt.rcParams['font.family'] = 'MS Gothic'

# 1225
markets = ["AAVE","ADA","ALGO","ALT","ATOM","AVAX","BAL","BCH","BNB","BRZ","BSV","BTC","BTMX","CHZ","COMP","CREAM","DEFI","DOGE","DOT","DRGN","EOS","ETH","EXCH","FIL","GRT","LINK","LTC","MID","OKB","PAXG","PRIV","SHIT","SOL","SUSHI","SXP","THETA","TRX","TRYB","UNI","UNISWAP","USDT","WAVES","XAUT","XRP","XTZ","YFI"]
# 0326
markets = ["1INCH","AAVE","ADA","ALGO","ALT","ATOM","AVAX","BAL","BCH","BNB","BRZ","BSV","BTC","BTMX","CHZ","COMP","CREAM","DEFI","DOGE","DOT","DRGN","EOS","ETH","EXCH","FIL","GRT","LINK","LTC","MID","OKB","OMG","PAXG","PRIV","SHIT","SUSHI","SXP","THETA","TRX","TRYB","UNI","UNISWAP","USDT","WAVES"]
axdict = {}
plt_index = 1

def req(url, params):
    txt = url + json.dumps(params)
    hash = hashlib.md5(txt.encode()).hexdigest()
    cache_name = './cache/'+hash+'.pickle'
    
    if (os.path.isfile(cache_name) and 'end_time' in params):
        with open(cache_name, mode='rb') as f:
            return pickle.load(f)
    
    res = requests.get(url, params=params)
    
    with open(cache_name, mode='wb') as f:
        pickle.dump(res, f)
    return res

fig = plt.figure(figsize=(30, 4*len(markets)), dpi=200)
for market in markets:
    """
    1225とPERPの価格の差をプロットする
    """
    url = 'https://ftx.com/api/markets/'+market+'-0326/candles'
    params = {'resolution': '3600', 'limit': '5000', 'end_time': '1609200000'}
    response = req(url, params)

    df = pd.DataFrame(response.json()["result"])
    df['startTime'] = pd.to_datetime(df['startTime'])
    limit = len(df)

    url = 'https://ftx.com/api/markets/'+market+'-PERP/candles'
    params = {'resolution': '3600', 'limit': limit, 'end_time': '1609200000'}
    response2 = req(url, params)

    df2 = pd.DataFrame(response2.json()["result"])
    df['diff'] = df['close'] - df2['close']
    df['diff_rate'] = ((df['diff'] * 2) / (df['close'] + df2['close']) ) * 100
#     if df.tail(1)["diff_rate"].item() < 0 or df.tail(1)["diff_rate"].item() > 1.5:
#         continue
        
    # 見た目
    axdict[market] = fig.add_subplot(len(markets), 1, plt_index)
    axdict[market].grid(True)

    # データセット
    axdict[market].plot(pd.to_datetime(df['startTime']), df['diff_rate'], label=market)
    axdict[market].set_title(market)
    
    plt_index += 1
    
plt.show()
